### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        '''
        Gini impurity:
        H(R) = 1 - p1**2 - p2**2 - ... - pK**2
        pk -- percent of k-th class in current node
        K -- number of classes
        '''
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) + (
            r_c ** 2) / (r_s * (l_s + r_s)), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        '''
        Entropy criteria:
        H(R) = -p1 * log2(p1) - ... - pK * log2(pK)
        pk -- percent of k-th class in current node
        K -- number of classes
        '''
        return -np.sum(l_c/l_s*np.log2(l_c/l_s), axis=1)-np.sum(r_c/r_s*np.log2(r_c/r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        '''
        Missclassification criteria:
        H(R) = 1 - max{p1, p2, ..., pK}
        pk -- percent of k-th class in current node
        K -- number of classes
        '''
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(
            r_c / (l_s + r_s), axis=1)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sort, y_sort=self.__sort_samples(x,y)
        class_num=self.num_class
        cut = np.int(self.min_samples_split / 2 - 1)
        y_sort_slit = y_sort[cut:-cut] if cut != 0 else y_sort
        r_border_ids = np.where(y_sort_slit[:-1] != y_sort_slit[1:])[0] + (cut + 1)
        if len(r_border_ids) == 0:
            return np.inf, None
        eq_el_count = r_border_ids - np.append([cut], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_num))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     y_sort[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y_sort[:cut], minlength=class_num)
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y_sort, minlength=class_num) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = y_sort.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        indx = np.argmin(gs)
        left_el_id = l_sizes[indx][0]
        return gs[indx], (x_sort[left_el_id - 1] + x_sort[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and self.max_depth<=depth) or \
            y.size < self.min_samples_split or \
            self.sufficient_share <= np.bincount(y).argmax() / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
        threshold = np.array([self.__find_threshold(x[:, i], y) for i in self.get_feature_ids(x.shape[1])])
        best_id = threshold[:, 0].argmin()
        best_threshold = threshold[best_id, 1]
        if best_threshold == None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
        else:
            x_l, x_r, y_l, y_r =self.__div_samples(x, y, best_id, best_threshold)
            if x_l.size == 0 or x_r.size == 0:
                self.tree[node_id] = (self.LEAF_TYPE,np.bincount(y).argmax(),np.bincount(y).astype(float) / y.size)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE,best_id, best_threshold)
                self.__fit_node(x_l, y_l,2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
            if self.G_function == self.__gini:
                g=(1 - np.sum(np.unique(y, return_counts=True)[1]** 2 / y.size ** 2))
                g_l=(1 - np.sum(np.unique(y_l, return_counts=True)[1] ** 2 / y_l.size ** 2))
                g_r=(1 - np.sum(np.unique(y_r, return_counts=True)[1] ** 2 / y_r.size ** 2))
                self.feature_importances_[best_id] += y.size*g - y_l.size*g_l - y_r.size*g_r
            if self.G_function == self.__entropy:
                g=-(np.sum((np.unique(y, return_counts=True)[1] / y.size) * np.log2(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=-(np.sum((np.unique(y_l, return_counts=True)[1] / y_l.size) * np.log2(np.unique(y_l, return_counts=True)[1] / y_l.size)))
                g_r=-(np.sum((np.unique(y_r, return_counts=True)[1] / y_r.size) * np.log2(np.unique(y_r, return_counts=True)[1] / y_r.size)))
                self.feature_importances_[best_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)   
            elif self.G_function == self.__misclass:
                g=(1 - np.max(np.sum(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=(1 - np.max(np.sum(np.unique(y_l, return_counts=True)[1] / y.size)))
                g_r=(1 - np.max(np.sum(np.unique(y_r, return_counts=True)[1] / y.size)))
                self.feature_importances_[best_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)

    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def feature_importance(self):
        features = []
        for i in self.tree.values():
            if i[0] == 0:
                features.append(i[1])
        feature = np.asarray(features)
        return np.bincount(feature).argsort()[:10]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

my_clf.fit(X_train, y_train)
my_clf.max_depth
my_clf.feature_importance()
#my_clf.tree

array([ 2,  3,  5,  7,  8, 10, 11,  1,  4, 12])

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 17.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 38.5 ms, sys: 856 µs, total: 39.3 ms
Wall time: 40.6 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.890993265993266

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251')
df

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,10.0,5,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8374,552,22.0,1,44,2,21,22,13,10.0,4,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8375,552,22.0,1,44,2,21,22,19,10.0,10,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8376,552,22.0,1,44,2,21,22,3,10.0,16,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0


In [10]:
for i in df.columns:
    if df[i].isnull().sum() > 1 / 10 *  df.shape[0]:
        df.drop(i,axis=1, inplace=True)
df

,iid,id,gender,idg,condtn,wave,round,position,order,partner,...,amb3_1,dec,attr,sinc,intel,fun,amb,like,prob,met
0,1,1.0,0,1,1,1,10,7,4,1,...,7.0,1,6.0,9.0,7.0,7.0,6.0,7.0,6.0,2.0
1,1,1.0,0,1,1,1,10,7,3,2,...,7.0,1,7.0,8.0,7.0,8.0,5.0,7.0,5.0,1.0
2,1,1.0,0,1,1,1,10,7,10,3,...,7.0,1,5.0,8.0,9.0,8.0,5.0,7.0,NaN,1.0
3,1,1.0,0,1,1,1,10,7,5,4,...,7.0,1,7.0,6.0,8.0,7.0,6.0,7.0,6.0,2.0
4,1,1.0,0,1,1,1,10,7,7,5,...,7.0,1,5.0,6.0,7.0,7.0,6.0,6.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,5,18,...,7.0,0,3.0,5.0,5.0,5.0,NaN,2.0,5.0,0.0
8374,552,22.0,1,44,2,21,22,13,4,19,...,7.0,0,4.0,6.0,8.0,4.0,4.0,4.0,4.0,0.0
8375,552,22.0,1,44,2,21,22,19,10,20,...,7.0,0,4.0,7.0,8.0,8.0,8.0,6.0,5.0,0.0
8376,552,22.0,1,44,2,21,22,3,16,21,...,7.0,0,4.0,6.0,5.0,4.0,NaN,5.0,5.0,0.0


In [11]:
for i in df.columns:
    if isinstance(df[i][0], str):
        df.drop(i,axis=1, inplace=True)
df

,iid,id,gender,idg,condtn,wave,round,position,order,partner,...,amb3_1,dec,attr,sinc,intel,fun,amb,like,prob,met
0,1,1.0,0,1,1,1,10,7,4,1,...,7.0,1,6.0,9.0,7.0,7.0,6.0,7.0,6.0,2.0
1,1,1.0,0,1,1,1,10,7,3,2,...,7.0,1,7.0,8.0,7.0,8.0,5.0,7.0,5.0,1.0
2,1,1.0,0,1,1,1,10,7,10,3,...,7.0,1,5.0,8.0,9.0,8.0,5.0,7.0,NaN,1.0
3,1,1.0,0,1,1,1,10,7,5,4,...,7.0,1,7.0,6.0,8.0,7.0,6.0,7.0,6.0,2.0
4,1,1.0,0,1,1,1,10,7,7,5,...,7.0,1,5.0,6.0,7.0,7.0,6.0,6.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,5,18,...,7.0,0,3.0,5.0,5.0,5.0,NaN,2.0,5.0,0.0
8374,552,22.0,1,44,2,21,22,13,4,19,...,7.0,0,4.0,6.0,8.0,4.0,4.0,4.0,4.0,0.0
8375,552,22.0,1,44,2,21,22,19,10,20,...,7.0,0,4.0,7.0,8.0,8.0,8.0,6.0,5.0,0.0
8376,552,22.0,1,44,2,21,22,3,16,21,...,7.0,0,4.0,6.0,5.0,4.0,NaN,5.0,5.0,0.0


In [12]:
for i , row in df.iterrows():
    if row.isnull().sum() > 0:
        df.drop(i, axis=0, inplace=True)
df

,iid,id,gender,idg,condtn,wave,round,position,order,partner,...,amb3_1,dec,attr,sinc,intel,fun,amb,like,prob,met
30,4,4.0,0,7,1,1,10,6,3,1,...,8.0,0,4.0,10.0,8.0,5.0,8.0,6.0,7.0,2.0
31,4,4.0,0,7,1,1,10,6,2,2,...,8.0,0,8.0,7.0,8.0,10.0,7.0,8.0,1.0,1.0
33,4,4.0,0,7,1,1,10,6,4,4,...,8.0,1,8.0,10.0,7.0,10.0,7.0,8.0,10.0,1.0
34,4,4.0,0,7,1,1,10,6,6,5,...,8.0,0,6.0,9.0,8.0,9.0,8.0,7.0,7.0,2.0
35,4,4.0,0,7,1,1,10,6,5,6,...,8.0,0,5.0,10.0,8.0,4.0,8.0,4.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,552,22.0,1,44,2,21,22,8,21,13,...,7.0,0,3.0,4.0,6.0,4.0,4.0,4.0,5.0,0.0
8369,552,22.0,1,44,2,21,22,4,17,14,...,7.0,0,3.0,3.0,9.0,6.0,9.0,6.0,3.0,0.0
8370,552,22.0,1,44,2,21,22,5,18,15,...,7.0,0,2.0,7.0,9.0,8.0,7.0,6.0,6.0,0.0
8372,552,22.0,1,44,2,21,22,17,8,17,...,7.0,0,7.0,5.0,5.0,5.0,6.0,4.0,4.0,0.0


## Проверка скорости работы на Speed Dating Data 

In [13]:
y = df.loc[:, 'gender'].values
X = df.drop('gender', axis=1).iloc[:,0:].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 87.4 ms, sys: 0 ns, total: 87.4 ms
Wall time: 93.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.3 s, sys: 3.18 ms, total: 1.3 s
Wall time: 1.34 s


## Проверка качества работы на Speed Dating Data

In [16]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9946178686759957

In [17]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9773933720133907

## Задание 3

In [18]:
fs = my_clf.feature_importance()
print(*df.columns[fs], sep=', ')

goal, amb_o, like_o, prob_o, age, field_cd, imprace, imprelig, fun3_1, go_out


## Задание 4

## Задание 5